**1. Installing Dependencies**

In [ ]:
pip install langchain openai faiss-cpu pypdf tiktoken streamlit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentence-transformers


In [ ]:
!pip install -U langchain-community


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os

pdf_path = list(uploaded.keys())[0]
print(f"Uploaded file: {pdf_path}")


In [ ]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def split_text_into_chunks(text, chunk_size=500, overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return splitter.split_text(text)

text = extract_text_from_pdf(pdf_path)
chunks = split_text_into_chunks(text)

print(f"Total chunks created: {len(chunks)}")
print(chunks[0])  # Preview first chunk


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# Convert chunks to LangChain Document format
documents = [Document(page_content=chunk) for chunk in chunks]

# Use local HuggingFace embeddings instead of OpenAI
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create FAISS index
vectorstore = FAISS.from_documents(documents, embedding_model)

# Save the vectorstore to Google Drive for persistence
save_path = "/content/drive/MyDrive/faiss_index"
vectorstore.save_local(save_path)
print(f"Vectorstore saved locally at {save_path}")

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.chains import RetrievalQA

In [ ]:
# Create text2text generation pipeline with a public Flan T5 model
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256,
    do_sample=False,
)

llm = HuggingFacePipeline(pipeline=generator)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

query = "What is the main topic of the document?"
answer = qa_chain.run(query)
print(answer)

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken here (replace with your actual token)
NGROK_AUTH_TOKEN = "2xODGEJKoPF8KK1erDrewYmIRLp_61D7mQKHB3ufn6GweXLtM"

# Authenticate ngrok with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Now start your Streamlit app and open the ngrok tunnel as before
import os
os.system('streamlit run app.py &')

public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")


In [ ]:
!pkill -f ngrok

